In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.9 MB/s eta 0:00:00


In [2]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 31.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.utils import shuffle
import torch
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, Trainer, TrainingArguments,BertTokenizerFast,RobertaForSequenceClassification,RobertaTokenizerFast
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss
from scipy.special import softmax
from nlp import load_dataset
from nlp import Dataset
import numpy as np

In [4]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [5]:
training_args1 = TrainingArguments(
    output_dir='/content/results', #存储结果文件的目录
    overwrite_output_dir=True,
    num_train_epochs=4, #4
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32, 
    learning_rate=5e-5,
    load_best_model_at_end=True, 
    metric_for_best_model = "accuracy", # 最后载入最优模型的评判标准，这里选用acc最高的那个模型参数
    weight_decay=0.01,
    warmup_steps=1,
    evaluation_strategy="steps", #这里设置每100个batch做一次评估，也可以为“epoch”，也就是每个epoch进行一次
    logging_strategy = "steps",
    save_strategy ='steps',
    logging_steps = 1,
    seed = 2023,
    logging_dir='/content/logs' #存储logs的目录
)
training_args2 = TrainingArguments(
    output_dir='/content/results', #存储结果文件的目录
    overwrite_output_dir=True,
    num_train_epochs=1, #2
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32, 
    learning_rate=5e-5,
    load_best_model_at_end=True, 
    metric_for_best_model = "accuracy", # 最后载入最优模型的评判标准，这里选用acc最高的那个模型参数
    weight_decay=0.01,
    warmup_steps=1,
    evaluation_strategy="steps", #这里设置每100个batch做一次评估，也可以为“epoch”，也就是每个epoch进行一次
    logging_strategy = "steps",
    save_strategy ='steps',
    logging_steps = 1,
    seed = 2023,
    logging_dir='/content/logs' #存储logs的目录
)

training_args = [training_args1,training_args2]

In [6]:
def tokenize_data(X_train,y_train):
  train_dataset = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=max_len)
  max_token = []
  for i in range(len(X_train.tolist())):
    decoded_encodings=tokenizer.tokenize(X_train.tolist()[i])
    max_token.append(len(decoded_encodings))
  print(max(max_token))
  train_dataset['label'] = y_train.values
  return train_dataset

In [7]:
def reply(index_hard,index_simple,train_data):
    change = 0
    for i in index_hard:
      pred_train_label = 0
      count = 0
      while pred_train_label == 0 and count <= 2:
        #print('gen harder',count,'round for data',i)
        answer = support_hard.loc[i,'hard'+str(count)]
        pred_train_label = train_bert(answer,None,None,'test',None) #get the predict label for specific round
        count += 1
      #print(count)
      if count < 3:
        train_data.loc[i,'content'] = answer
        #print(train_data.loc[i,'label'])
        change += 1
      if change >= np.round(len(index_hard)/2):
        break
    print('changing',change,'to hard.')
    change = 0
    for i in index_simple:
      pred_train_label = 1
      count = 0
      while pred_train_label == 1 and count <= 2:
        #print('gen simpler',count,'round for data',i)
        answer = support_simple.loc[i,'simple'+str(count)]
        pred_train_label = train_bert(answer,None,None,'test',None) #get the predict label for specific round
        count += 1
      if count < 3:
        train_data.loc[i,'content'] = answer
        change += 1
      if change >= np.round(len(index_simple)/2):
        break
    print('changing',change,'to simple.')

In [8]:
train_data = pd.read_csv('/content/data_law_read.csv',encoding='gb18030') #Read data
init_re = pd.read_csv('/content/init_re.csv',encoding='gb18030')
support_hard = pd.read_csv('/content/harder.csv',encoding='gb18030')
support_simple = pd.read_csv('/content/simpler.csv',encoding='gb18030')
init_re['label']=0
init_re['index_article'] = train_data['index_article']
init_re['index_paragraph'] = train_data['index_paragraph']
train_data = pd.concat([init_re,train_data]).reset_index()

In [9]:
val_data = pd.read_csv('/content/val_law_data.csv',encoding='gb18030')#need to gen before
test_data = pd.read_csv('/content/test_law_data.csv',encoding='gb18030')#need to gen before
test_data = test_data.dropna()
seed = 2023
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

X_val = val_data['content'] 

y_val = val_data['label']
#X_train, X_val, y_train, y_val = train_test_split(train_data['content'],train_data['label'], test_size=0.1, random_state=2023)
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=2,output_hidden_states=False)
max_len = 216
val_dataset = tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=max_len)
val_dataset['label'] = y_val.values
test_dataset = tokenizer(test_data['content'].tolist(),padding=True, truncation=True, max_length=max_len)
val_dataset = Dataset.from_dict(val_dataset)
test_dataset = Dataset.from_dict(test_dataset)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [10]:
def train_bert(train_dataset,val_dataset,test_dataset,status,training_args):
  global max_len
  trainer = Trainer(model=model,args=training_args,compute_metrics=compute_metrics,train_dataset=train_dataset,eval_dataset=val_dataset,tokenizer=tokenizer)
  if status == 'train':
    trainer.train()
    pred_train = trainer.predict(test_dataset=train_dataset).predictions
    pred = trainer.predict(test_dataset=test_dataset).predictions
    prob_train = softmax(pred_train, axis = 1)
    prob = softmax(pred, axis = 1)
    pred_label_train = pred_train.argmax(-1)
    pred_label = prob.argmax(-1)
    return pred_label_train,pred_label
  else:
    df = pd.DataFrame(columns=['content'], index=[0])
    df.loc[0,'content'] = train_dataset
    train_dataset = tokenizer(df['content'].tolist(),padding=True,truncation=True,max_length=max_len) #input train dataset is an paragraph
    train_dataset = Dataset.from_dict(train_dataset)
    pred_train = trainer.predict(test_dataset=train_dataset).predictions
    prob_train = softmax(pred_train, axis = 1)
    pred_label_train = pred_train.argmax(-1)
    return pred_label_train


In [11]:
def judge(y_train,y_train_pred):
  index_hard = []
  index_simple = []
  for i in range(len(y_train)):
    if y_train[i] == y_train_pred[i] and y_train[i] == 0:
      index_hard.append(i)
    elif y_train[i] != y_train_pred[i] and y_train[i] == 0:
      index_simple.append(i)
  return index_hard,index_simple


In [12]:
train_data = train_data.dropna()
X_train = train_data['content']
y_train = train_data['label']
train_dataset = tokenize_data(X_train,y_train)
train_dataset = Dataset.from_dict(train_dataset)
train_label,test_label = train_bert(train_dataset,val_dataset,test_dataset,'train',training_args[0])
acc = accuracy_score(test_data['label'], test_label)
precision, recall, f1, _ = precision_recall_fscore_support(test_data['label'], test_label, average='binary')
print('in round 1 testing acc is:',acc)
print(precision,recall,f1)
index_hard, index_simple = judge(train_data['label'],train_label)
reply(index_hard,index_simple,train_data)
train_data = train_data.dropna()
X_train = train_data['content']
y_train = train_data['label']
train_dataset = tokenize_data(X_train,y_train)
train_dataset = Dataset.from_dict(train_dataset)
train_label,test_label = train_bert(train_dataset,val_dataset,test_dataset,'train',training_args[1])
acc = accuracy_score(test_data['label'], test_label)
precision, recall, f1, _ = precision_recall_fscore_support(test_data['label'], test_label, average='binary')
print('in round 2 testing acc is:',acc)
print(precision,recall,f1)
test_data['pred']=test_label
test_data.to_csv('/content/predict2_law_data.csv')

344


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.688900,0.715391,0.500000,0.666667,0.500000,1.000000
2,0.728100,0.707664,0.500000,0.666667,0.500000,1.000000
3,0.728600,0.699148,0.500000,0.666667,0.500000,1.000000
4,0.699600,0.693391,0.500000,0.666667,0.500000,1.000000
5,0.696900,0.690728,0.622951,0.439024,0.857143,0.295082
6,0.662300,0.689166,0.500000,0.000000,0.000000,0.000000
7,0.667800,0.690863,0.500000,0.000000,0.000000,0.000000
8,0.687500,0.689528,0.500000,0.000000,0.000000,0.000000
9,0.702300,0.684109,0.500000,0.000000,0.000000,0.000000
10,0.685900,0.676503,0.500000,0.000000,0.000000,0.000000


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

in round 1 testing acc is: 0.8350515463917526
0.9452054794520548 0.711340206185567 0.8117647058823529


changing 6 to hard.
changing 0 to simple.
344


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.092200,0.262274,0.926230,0.923077,0.964286,0.885246
2,0.121200,1.027343,0.762295,0.688172,1.000000,0.524590
3,0.829200,0.420198,0.885246,0.870370,1.000000,0.770492
4,0.173700,0.188346,0.934426,0.937500,0.895522,0.983607
5,0.451200,0.202670,0.934426,0.937500,0.895522,0.983607
6,0.285100,0.237575,0.918033,0.915254,0.947368,0.885246
7,0.098900,0.564504,0.770492,0.708333,0.971429,0.557377
8,0.262300,0.635894,0.770492,0.708333,0.971429,0.557377
9,0.209000,0.450184,0.795082,0.747475,0.973684,0.606557
10,0.361800,0.245276,0.901639,0.894737,0.962264,0.836066


in round 2 testing acc is: 0.8505154639175257
0.9594594594594594 0.7319587628865979 0.8304093567251462


In [13]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 25.2 MB/s eta 0:00:00
